
# Conditional autoregressive transformer

Train a transformer to predict missing pixel from mnist 

### plan

* note to try padded mnist (relative encoding might require black padding???)
* probably don't need positional encoding?
* create transformer model
* masking 
* randomised masking
* relative position encoding (x - current_x, y - current_y, val)
* train to predict when current pixel missing
* train to predict when 10% are missing
* train to predict when 90% are missing
* train to predict when 99% are missing



In [1]:
import tensorflow as tf
import tensorflow.keras as keras

from PIL import Image, ImageDraw
from IPython.display import display
import numpy as np
import itertools

def display_uint8_image(image):
    if len(image.shape) == 3:
        image = image[:, :, 0]
    if tf.is_tensor(image):
        image = image.numpy()
    display(Image.fromarray(image, "L"))

def display_float32_image(image):
    if len(image.shape) == 3:
        image = image[:, :, 0]
    if tf.is_tensor(image):
        image = image.numpy()
    display_uint8_image(image.astype(np.uint8))





In [2]:
# reformat MNIST data into (x, y, val) tuples

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(
    path='mnist.npz'
)
# [row, col, value] tuples
def mnist_to_rcv_tuples(dataset):
    # positional encoding is just indices as floats
    x_idx = np.arange(dataset.shape[1]).astype(np.float32)
    y_idx = np.arange(dataset.shape[2]).astype(np.float32)

    # make 28x28 cartesian product and repeat N times
    idxs = np.transpose([np.repeat(y_idx, len(x_idx)),np.tile(x_idx, len(y_idx))])
    
    all_idxs = np.tile(idxs.reshape(-1), dataset.shape[0]).reshape(-1,784,2)
    # pixel data as float between 0 and 255
    all = dataset.astype(np.float32).reshape(-1, 784, 1)
    
    big = np.dstack((all_idxs, all))
    
    return big

def random_offset_tuples(dataset, n_masked_px=1):
    tuples = mnist_to_rcv_tuples(dataset)
    offsets = np.random.randint(0, 28, [dataset.shape[0], 2])
    tuples[:, :, 0] -= offsets[:, 0, np.newaxis] # row (y) offset
    tuples[:, :, 1] -= offsets[:, 1, np.newaxis] # col (x) offset
    mask_idxs = offsets[:, 0] * 28 + offsets[:, 1]
    mask_idxs = mask_idxs[:, np.newaxis] # support multiple idxs
    if n_masked_px > 1:
        multiple_mask_idxs = np.random.randint(0, 28, [dataset.shape[0], n_masked_px-1, 2])
        multiple_mask_idxs = multiple_mask_idxs[:, :, 0] * 28 + multiple_mask_idxs[:, :, 1]
        mask_idxs = np.concatenate([multiple_mask_idxs, mask_idxs], axis=1)
    target_pxs = dataset[np.arange(dataset.shape[0]), offsets[:, 0], offsets[:, 1]].astype(np.float32)
    return { "imgs": dataset, "tuples": tuples, "offsets": offsets, "target_pxs": target_pxs, "mask_idxs": mask_idxs }

dataset1 = random_offset_tuples(x_train)
dataset2 = random_offset_tuples(x_train, n_masked_px=400)
dataset3 = random_offset_tuples(x_train, n_masked_px=700)

print(dataset1["mask_idxs"][2])
print(dataset2["mask_idxs"][2])
print(dataset3["mask_idxs"][2])

# should be the same
display_uint8_image(x_train[0])
display_uint8_image(x_train[2])

print(dataset1["tuples"][:100, :])

[659]
[ 88 676 746 659 399 376 156 415 605 553 565  41 502  45 529 433 333 149
 260 610 408 251  35 104 777 361 420 436  41 722  94 527  50  40 692 408
 288 660 375 446  92 136  46 213 263 309 746  31 221  33 327 339 732  94
 555   3 272 229 733 134 557 370 436 699 523 167  14 210 553 265 343 701
 725 308  44 742 236 319 120 403 672 584 213 390 546  99 184 162 770 299
  22  47 119 666 335 750 511  54 494 285 759 532 102 293 604 237 754 562
 685 339 171 207 385 374 340 412 464 450  86 313 340 389 256 714 333 447
 555 459 762 762 574 291 220 301 169 434 500 212 519 684 486 321 755  89
 631 597 399  11 583 147 688 470   0 506 739 661 642 266 243 421 781 647
 676 319  98 512 678 120 408  88 349 604 451 586 572 391  13 326 577 590
 284 490 578  35 212 656 315  31 492  80  21 695 335 470 778 280 180  21
 231 328 165 413 409 387 175 357 389 192 398 655 689 105 664  96 351 365
 555 103 490 266 671 151 308 380 464 619 709   2 307  73 605 463 211 522
 400 709  40   7 703 291 493 475 357 622 127 

[[[ -7.  -4.   0.]
  [ -7.  -3.   0.]
  [ -7.  -2.   0.]
  ...
  [ 20.  21.   0.]
  [ 20.  22.   0.]
  [ 20.  23.   0.]]

 [[ -8. -13.   0.]
  [ -8. -12.   0.]
  [ -8. -11.   0.]
  ...
  [ 19.  12.   0.]
  [ 19.  13.   0.]
  [ 19.  14.   0.]]

 [[-23. -15.   0.]
  [-23. -14.   0.]
  [-23. -13.   0.]
  ...
  [  4.  10.   0.]
  [  4.  11.   0.]
  [  4.  12.   0.]]

 ...

 [[ -3. -15.   0.]
  [ -3. -14.   0.]
  [ -3. -13.   0.]
  ...
  [ 24.  10.   0.]
  [ 24.  11.   0.]
  [ 24.  12.   0.]]

 [[-16. -20.   0.]
  [-16. -19.   0.]
  [-16. -18.   0.]
  ...
  [ 11.   5.   0.]
  [ 11.   6.   0.]
  [ 11.   7.   0.]]

 [[-25.  -4.   0.]
  [-25.  -3.   0.]
  [-25.  -2.   0.]
  ...
  [  2.  21.   0.]
  [  2.  22.   0.]
  [  2.  23.   0.]]]


In [3]:

print(dataset1["tuples"].shape, dataset1["offsets"].shape, dataset1["target_pxs"].shape)
print(dataset1["tuples"][0])
print(dataset1["offsets"][0])
print(dataset1["target_pxs"][0])

# actual training dataset is 784 x,y,v tuples, but with the spatial location changed so the the pixel to predict is in the center
# and the pixel value to predict removed (via masking)

# then, can mask more and more stuff and compare performance

# we can also use the model in an "iterative" mode to generate images, in an arbitrary order.

(60000, 784, 3) (60000, 2) (60000,)
[[-7. -4.  0.]
 [-7. -3.  0.]
 [-7. -2.  0.]
 ...
 [20. 21.  0.]
 [20. 22.  0.]
 [20. 23.  0.]]
[7 4]
0.0


In [4]:
from tensorflow import keras
from tensorflow.keras import layers

In [5]:
# attention masking test

def idxs_to_onehots(idxs):
    onehots = tf.one_hot(idxs, depth=784, dtype=tf.bool, on_value=False, off_value=True)
    return onehots

# takes 2D tensor (batch and index list)
def idxs_to_multihot(idxs):
    onehots = idxs_to_onehots(idxs)
    multihot = tf.reduce_all(onehots, axis=1)
    return multihot

def idxs_to_attention_mask(idxs):
    multihot = idxs_to_multihot(idxs)
    attn_mask = tf.logical_and(multihot[:, :, None], multihot[:, None, :])
    return attn_mask

def idxs_to_image_mask(idxs):
    multihot = idxs_to_multihot(idxs)
    multihot = tf.reshape(multihot, [-1, 28, 28])
    return multihot

print("mask_idxs shape     ", dataset2["mask_idxs"][:1].shape)
print("onehot shape        ", idxs_to_onehots(dataset2["mask_idxs"][:1]).shape)
print("multihot shape      ", idxs_to_multihot(dataset2["mask_idxs"][:1]).shape)
print("attention_mask shape", idxs_to_attention_mask(dataset2["mask_idxs"][:1]).shape)
print("image_mask shape    ", idxs_to_image_mask(dataset2["mask_idxs"][:1]).shape)


mask_idxs shape      (1, 400)
onehot shape         (1, 400, 784)
multihot shape       (1, 784)
attention_mask shape (1, 784, 784)
image_mask shape     (1, 28, 28)


2021-08-26 12:56:19.555969: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-26 12:56:19.564505: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-26 12:56:19.565091: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-26 12:56:19.566035: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
202

In [6]:
# example from keras docs https://keras.io/examples/nlp/text_classification_with_transformer/
# uses built-in MultiHeadAttention https://keras.io/api/layers/attention_layers/multi_head_attention/

class TransformerBlock(layers.Layer):
    def __init__(self, n_embed_dim, n_heads, n_dense_dim, dropout_rate, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.att = layers.MultiHeadAttention(num_heads=n_heads, key_dim=n_embed_dim)
        self.dense_net = keras.Sequential([
            layers.Dense(n_dense_dim, activation='relu'),
            layers.Dense(n_embed_dim),
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(dropout_rate)
        self.dropout2 = layers.Dropout(dropout_rate)
    
    def call(self, inputs, mask_idxs, is_training):
#         print("mask_idxs shape", mask_idxs.shape)
        mask = tf.one_hot(mask_idxs, depth=inputs.shape[1], on_value=False, off_value=True, dtype=tf.bool)
#         print("onehot shape", mask.shape)
        mask = tf.reduce_all(mask, axis=1)
#         print("multihot shape", mask.shape)
        # make mask 2D
        mask = tf.logical_and(mask[:, :, None], mask[:, None, :])
#         print("squaremultihot shape", mask.shape)
        attn_output = self.att(inputs, inputs, attention_mask=mask)
        attn_output = self.dropout1(attn_output, training=is_training)
        out1 = self.layernorm1(inputs + attn_output)
        dense_output = self.dense_net(out1)
        dense_output = self.dropout2(dense_output, training=is_training)
        return self.layernorm2(out1 + dense_output)

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions



In [7]:
tf.constant([True, False]).dtype

tf.bool

In [8]:
def model(batch_size):

    # no batch size to start makes it simpler
    n_embd = 10

    data_input = keras.Input(shape=[784, 3], batch_size=None)
    mask_input = keras.Input(shape=[None], batch_size=None, dtype=tf.int32)
    m = data_input

    # make n_embd-dimensional input embeddings per pixel from [x, y, v]
    m = layers.Dense(20, activation='relu')(m)
    m = layers.Dense(20, activation='relu')(m)
    m = layers.Dense(n_embd, activation=None)(m)
    m = TransformerBlock(n_embed_dim=n_embd, n_heads=7, n_dense_dim=16, dropout_rate=0.1, name="txform1")(m, mask_idxs=mask_input, is_training=True)
    m = TransformerBlock(n_embed_dim=n_embd, n_heads=7, n_dense_dim=16, dropout_rate=0.1, name="txform2")(m, mask_idxs=mask_input, is_training=True)
    m = layers.Flatten()(m)
    # m = layers.Dense(10, activation='relu')(m)
    m = layers.Dense(1, activation=None)(m)

    model = keras.Model(inputs=[data_input, mask_input], outputs=[m])

    return model


In [10]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Wed_Jun__2_19:15:15_PDT_2021
Cuda compilation tools, release 11.4, V11.4.48
Build cuda_11.4.r11.4/compiler.30033411_0
Thu Aug 26 12:56:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 470.63.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:26:00.0  On |                  N/A |
|  0%   40C    P5    17W / 170W |   4840MiB /  5926MiB |     41%      Default |
|                               |       

In [11]:
batch_size=16

def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
lr_metric = get_lr_metric(optimizer)

txformer = model(batch_size)
txformer.compile(optimizer=optimizer, loss='mse', metrics=['MeanAbsoluteError', lr_metric])
load_saved = True
if load_saved:
    txformer.load_weights("./models/txformer")

txformer.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 784, 3)]     0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 784, 20)      80          input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 784, 20)      420         dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 784, 10)      210         dense_1[0][0]                    
______________________________________________________________________________________________

In [12]:
def fit_one_epoch(dataset):
    txformer.fit(x=[dataset["tuples"], dataset["mask_idxs"]], y=dataset["target_pxs"], epochs=1, batch_size=batch_size)

# display:
# - before mask
# - mask
# - after mask
# - prediction
def gen_image(dataset, idx):
    img = np.zeros([28, 28])
    mask = dataset["mask_idxs"][idx]
    erow, ecol = dataset["offsets"][idx]

    image_mask = np.array(idxs_to_image_mask(mask[np.newaxis, :]), np.uint8)[0]
    masked_image = np.copy(dataset["imgs"][idx])
    display_uint8_image(masked_image) # before mask
    display_uint8_image(image_mask * 255) # mask
    masked_image = masked_image * image_mask
    display_uint8_image(masked_image) # after mask
    
    for row in range(28):
        for col in range(28):
            img_tups = np.copy(dataset["tuples"][idx])
            img_tups[:, 0] += erow
            img_tups[:, 1] += ecol
            img_tups[:, 0] -= row
            img_tups[:, 1] -= col
            mask_idx = row*28+col
            
            this_mask = np.concatenate([mask, np.array([mask_idx])])
            px = txformer([img_tups[np.newaxis, :, :], this_mask[np.newaxis, :]])
            img[row, col] = np.clip(px[0, 0], 0, 255)
    print("# px masked out:", len(this_mask))
    display_float32_image(img)

gen_image(dataset1, 2)
gen_image(dataset2, 2)
gen_image(dataset3, 2)

# px masked out: 2


# px masked out: 401


# px masked out: 701


In [15]:
fit_one_epoch(dataset1)
fit_one_epoch(dataset1)

2021-08-25 17:26:40.113714: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


3750/3750 [==============================] - 238s 64ms/step - loss: 924.8143 - mean_absolute_error: 14.0071 - lr: 9.9998e-04


In [16]:
gen_image(dataset1, 2)
gen_image(dataset2, 2)
gen_image(dataset3, 2)

(1,)
(28, 28)


(400,)
(28, 28)


(700,)
(28, 28)


In [17]:
fit_one_epoch(dataset2)

3750/3750 [==============================] - 242s 64ms/step - loss: 1759.8699 - mean_absolute_error: 23.5656 - lr: 9.9998e-04


In [18]:
gen_image(dataset1, 2)
gen_image(dataset2, 2)
gen_image(dataset3, 2)

(1,)
(28, 28)


(400,)
(28, 28)


(700,)
(28, 28)


In [19]:
fit_one_epoch(dataset3)

3750/3750 [==============================] - 245s 65ms/step - loss: 1393.4340 - mean_absolute_error: 19.5466 - lr: 9.9998e-04


In [20]:
gen_image(dataset1, 2)
gen_image(dataset2, 2)
gen_image(dataset3, 2)

(1,)
(28, 28)


(400,)
(28, 28)


(700,)
(28, 28)


In [21]:
txformer.save_weights("./models/txformer")